In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(
  origin=dataset_url,
  fname='flower_photos',
  untar=True
)
data_dir = pathlib.Path(data_dir)

228813984/228813984 [==============================] - 6s 0us/step


In [ ]:
# Configurations

SEED = 0
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.20
EPOCHS = 10

In [ ]:
# Creating the training set

training_set = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
)

# Creating the validation set

validation_set = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [ ]:
categories = training_set.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip(
      "horizontal_and_vertical", 
      input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3)
  ),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.RandomCrop(
    128, 128
  ),
  tf.keras.layers.Resizing(IMAGE_SIZE[0], IMAGE_SIZE[0]),
  tf.keras.layers.RandomContrast(0.3)
])

def prepare(ds, shuffle=False, augment=False):

    if shuffle:
        ds = ds.shuffle(1000)

    # Use data augmentation only on the training set.
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y), 
            num_parallel_calls=AUTOTUNE
        )

    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(training_set, shuffle=True, augment=True)
val_ds = prepare(validation_set)

In [ ]:
tf.keras.backend.clear_session()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=3, 
    min_lr=0.0001
)

resnet_model = tf.keras.applications.resnet50.ResNet50(
    input_shape=IMAGE_SIZE+ (3,),
    include_top=False,
    weights='imagenet',
)

resnet_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = resnet_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(x)

transfer_resnet_model = tf.keras.Model(inputs, outputs)

transfer_resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

transfer_fine = transfer_resnet_model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[reduce_lr]
)

94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/5
92/92 [==============================] - 48s 365ms/step - loss: 0.9427 - accuracy: 0.6441 - val_loss: 0.4937 - val_accuracy: 0.8134 - lr: 0.0010
Epoch 2/5
92/92 [==============================] - 35s 329ms/step - loss: 0.6622 - accuracy: 0.7520 - val_loss: 0.4980 - val_accuracy: 0.8147 - lr: 0.0010
Epoch 3/5
92/92 [==============================] - 39s 379ms/step - loss: 0.5858 - accuracy: 0.7776 - val_loss: 0.4465 - val_accuracy: 0.8256 - lr: 0.0010
Epoch 4/5
92/92 [==============================] - 35s 331ms/step - loss: 0.5762 - accuracy: 0.7875 - val_loss: 0.5039 - val_accuracy: 0.8134 - lr: 0.0010
Epoch 5/5
92/92 [==============================] - 35s 332ms/step - loss: 0.5518 - accuracy: 0.7909 - val_loss: 0.4125 - val_accuracy: 0.8501 - lr: 0.0010


In [ ]:
len(transfer_resnet_model.trainable_variables)

4

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(resnet_model.layers))

Number of layers in the base model:  175


In [ ]:
resnet_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 151

# Freeze all the layers before the `fine_tune_at` layer
for layer in resnet_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
transfer_resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

len(transfer_resnet_model.trainable_variables)

32

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=3, 
    min_lr=0.00001
)

transfer_history = transfer_resnet_model.fit(
  train_ds,
  epochs=15,
  initial_epoch=transfer_fine.epoch[-1],
  validation_data=val_ds,
  callbacks=[reduce_lr]
)

Epoch 5/15
92/92 [==============================] - 41s 350ms/step - loss: 0.4869 - accuracy: 0.8161 - val_loss: 0.4535 - val_accuracy: 0.8501 - lr: 1.0000e-04
Epoch 6/15
92/92 [==============================] - 36s 345ms/step - loss: 0.4421 - accuracy: 0.8542 - val_loss: 0.4462 - val_accuracy: 0.8597 - lr: 1.0000e-04
Epoch 7/15
92/92 [==============================] - 37s 350ms/step - loss: 0.3999 - accuracy: 0.8580 - val_loss: 0.4278 - val_accuracy: 0.8638 - lr: 1.0000e-04
Epoch 8/15
92/92 [==============================] - 37s 347ms/step - loss: 0.3916 - accuracy: 0.8631 - val_loss: 0.4765 - val_accuracy: 0.8515 - lr: 1.0000e-04
Epoch 9/15
92/92 [==============================] - 38s 358ms/step - loss: 0.3308 - accuracy: 0.8856 - val_loss: 0.3980 - val_accuracy: 0.8801 - lr: 1.0000e-04
Epoch 10/15
92/92 [==============================] - 37s 351ms/step - loss: 0.3177 - accuracy: 0.8907 - val_loss: 0.3974 - val_accuracy: 0.8719 - lr: 1.0000e-04
Epoch 11/15
92/92 [====================

In [ ]:
import matplotlib.pyplot as plt

def plot_actual_prediction(model, categories, validation_set):

    plt.figure(figsize=(20, 10))
    for images, labels in validation_set.take(3):
        for i in range(45):
            ax = plt.subplot(9, 5, i + 1)
            
            img_array = images[i].numpy().astype("uint8")
            prediction = model.predict(np.array([img_array]))
            prediction_name = categories[np.argmax(prediction)]
            real_name = categories[np.argmax(labels[i])]
            
            plt.imshow(img_array)
            if prediction_name == real_name:
                plt.title(f'real: {real_name}\npred:{prediction_name}', fontdict={'color': 'g'})
            else:
                plt.title(f'real: {real_name}\npred:{prediction_name}', fontdict={'color': 'r'})
            
    plt.axis("off")
    plt.tight_layout();

In [ ]:
# plot_actual_prediction(transfer_resnet_model, categories, val_ds)

In [ ]:
from sklearn.metrics import classification_report

predicted_classes = np.argmax(transfer_resnet_model.predict(val_ds), axis=1)

23/23 [==============================] - 4s 100ms/step


In [ ]:
# predicted_classes

In [ ]:
# true_classes = np.argmax(tf.concat([y for x, y in val_ds], axis=0), axis=1)

In [ ]:
# true_classes

In [ ]:
# true_classes = val_ds.class_labels
true_classes = np.argmax(tf.concat([y for x, y in val_ds], axis=0), axis=1)
report = classification_report(true_classes, predicted_classes, target_names=categories)

In [ ]:
print(report)

              precision    recall  f1-score   support

       daisy       0.12      0.13      0.13       117
   dandelion       0.27      0.25      0.26       194
       roses       0.18      0.17      0.17       138
  sunflowers       0.15      0.15      0.15       137
      tulips       0.20      0.23      0.21       148

    accuracy                           0.19       734
   macro avg       0.18      0.18      0.18       734
weighted avg       0.19      0.19      0.19       734



In [ ]:
transfer_resnet_model.save("saved_model/transfer_model_2.h5")

In [ ]:
predictions = transfer_resnet_model.predict(val_ds)

23/23 [==============================] - 3s 93ms/step


In [ ]:
predictions[0]

array([1.1351188e-08, 1.0000000e+00, 5.0380178e-10, 9.6835662e-10,
       2.3197676e-11], dtype=float32)

In [ ]:
predictions[0]

array([1.1351188e-08, 1.0000000e+00, 5.0380178e-10, 9.6835662e-10,
       2.3197676e-11], dtype=float32)

In [ ]:
import pandas as pd
d = pd.DataFrame([categories, predictions[0]]).T
d.columns = ["Flower", "Prediction"]
d.sort_values(by='Prediction', inplace=True, ascending=False)
d

,Flower,Prediction
1,dandelion,1.0
0,daisy,0.0
3,sunflowers,0.0
2,roses,0.0
4,tulips,0.0
